# Table of Contents
* [1. Quick start with ECEI2D package](#1.-Quick-start-with-ECEI2D-package)
	* [1.1 Initial setup and importing ECEI2D modules](#1.1-Initial-setup-and-importing-ECEI2D-modules)
	* [1.2 Gaussian Antenna detector](#1.2-Gaussian-Antenna-detector)
	* [1.3 Single channel ECE2D diagnosis](#1.3-Single-channel-ECE2D-diagnosis)
		* [1.3.1 Diagnose with a user set mesh](#1.3.1-Diagnose-with-a-user-set-mesh)
		* [1.3.2 Automatically adjust stepsize](#1.3.2-Automatically-adjust-stepsize)
		* [1.3.3 Information about diagnosing locations](#1.3.3-Information-about-diagnosing-locations)
	* [1.4 Multi-channel ECE Imaging](#1.4-Multi-channel-ECE-Imaging)
* [2. Benchmarks and Error Assessment](#2.-Benchmarks-and-Error-Assessment)
	* [2.1 Higher harmonic effects](#2.1-Higher-harmonic-effects)
	* [2.2 Error analysis](#2.2-Error-analysis)


# 1. Quick start with ECEI2D package

## 1.1 Initial setup and importing ECEI2D modules

In [ ]:
# Some initial setups
from __future__ import print_function
import sys

import numpy as np
from scipy.integrate import trapz, cumtrapz
import numpy.fft as fft
import matplotlib.pyplot as plt
from matplotlib import rcParams

from FPSDP.GeneralSettings.UnitSystem import cgs
import FPSDP.Plasma.Analytical_Profiles.TestParameter as tp

%matplotlib inline

rcParams['figure.figsize'] = [12, 9]
rcParams['font.size'] = 18

c = cgs['c']
keV = cgs['keV']
e = cgs['e']
me = cgs['m_e']

# We will use a uniform Te profile to do the benchmarks
Te0 = 10*keV
ne0 = 2e13
tp.set_parameter2D(Te_0 = Te0, ne_0=ne0, Te_shape='uniform', ne_shape='Hmode')
p2d_uni = tp.create_profile2D()
p2d_uni.setup_interps()

`FPSDP.Diagnostics.ECEI.ECEI2D` package has mainly two useful modules: `Reciprocity` and `Detector2D`.
`Reciprocity` has class `ECE2D` which carries out a single ECE diagnostic process, while `Detector2D` provides classes for 2D detectors, among which `GaussianAntenna` might be the most commonly used. 

In [ ]:
import FPSDP.Diagnostics.ECEI.ECEI2D.Reciprocity as rcp
from FPSDP.Diagnostics.ECEI.ECEI2D.Detector2D import GaussianAntenna

## 1.2 Gaussian Antenna detector

`GaussianAntenna` simulates an antenna pattern that can be described as a Gaussian Beam <a name="ref-1"/>[(Svelto, 2010)](#cite-svelto2010principles). 
In addition to a single frequency, it accept a list of frequencies, a list of wave vectors, and a list of corresponding receiving(emitting) powers. This provides a fully customizable frequency bandwidth and shape. 
Any other keyword arguments passed in the constructor will be given to `FPSDP.Maths.LightBeam.GaussianBeam` constructor. In particular, three arguments are required: `waist_x`, `waist_y`, `w_0y`.  

In [ ]:
omega = 8e11
k = omega/c
# single frequency detector
detector = GaussianAntenna(omega_list=[omega], k_list=[k], power_list=[1], waist_x=175, waist_y=0, w_0y=2)

## 1.3 Single channel ECE2D diagnosis

`Reciprocity` module provides `ECE2D` class that represents a single channel diagnosis. 

It is initialized with a ECEI plasma profile, a `Detector2D` detector, a specified polarization, and 4 parameters determining the model used for current correlation tensor formula and dielectric tensor formula. Namely, they are `max_harmonic`, `max_power`, `weakly_relativistic` and `isotropic`. The first two are integers. They determine the highest harmonic frequency and highest order in FLR effect included in both current correlation tensor and dielectric tensor. The `weakly_relativistic` paramter is boolean, if `True`, weakly relativistic formula will be used, otherwise non-relativistic formula instead. `isotropic` is also boolean, `True` indicates the electron distribution is assumed isotropic, and the current correlation tensor can be easily obtained from anti-Hermitian part of corresponding dielectric tensor. 

> Right now, only isotropic plasma is implemented. 

In [ ]:
ece = rcp.ECE2D(plasma=p2d_uni, detector=detector, polarization='X', max_harmonic=2, max_power=2, 
                weakly_relativistic=True, isotropic=True)

### 1.3.1 Diagnose with a user set mesh

After initialization, a set of coordinates needs to be specified for ECE2D, as a initial mesh for preliminary calculations. `set_coords(coords)` method accepts a list of 1D coordinates, and will generate 3D mesh using for ECE diagnosis.

In [ ]:
X1D = np.linspace(250, 150, 200)
Y1D = np.linspace(-20, 20, 65)
Z1D = np.linspace(-20, 20, 65)

# set_coords needs to be called before running any other methods in ECE2D
ece.set_coords([Z1D, Y1D, X1D])

We can directly diagnose the plasma with this simple mesh, just call `diagnose` method. 

Three parameters are accepted in `diagnose`: `time`, `debug` and `auto_patch`. 
- `time`: integer, specifies the time step of plasma fluctuation that should be used for diagnosis, if not given, equilibrium plasma will be used.
- `debug`: bool, if True, a lot of extra information will be stored after diagnosis, for further investigation.
- `auto_patch`: bool, if True, program will try to optimize the grid in X direction, make finer mesh where emission is coming, and coarser mesh where wave propagates with no absorption or emission. A better way of doing this is calling `auto_adjust_mesh` method, with a tunable fine structure constant.

In [ ]:
# we diagnose the equilibrium plasma with no auto coordinates adjustment. Keep more information by setting debug=True
Te = ece.diagnose(debug=True)

In [ ]:
Te/keV

> Note that the diagnosed Te is smaller than real Te, which is 10keV. The reason is our preset coordinates in X is too coarse within the emission area, thus not sampling enough data. Integration error is large due to this under sample situation. The way to improve this is to call `auto_adjust_mesh` method.

We can take a look at where received emission is coming from by examine the `integrand_list`.

In [ ]:
emission_spot = ece.integrand_list[0]

In [ ]:
plt.imshow(np.real(emission_spot[0,:,:]), extent=[X1D[0], X1D[-1], Y1D[0], Y1D[-1]], origin='lower')

### 1.3.2 Automatically adjust stepsize

`ECE2D` has `auto_adjust_mesh` method that will automatically choose proper grid points on X direction. The way it does this is run a preliminary propagation on the original mesh, and calculate the actual emission locations that antenna sees. Make fine mesh within these regions, and setup coarser mesh outside them. 

Default parameters are:
- emission region boundaries: The emission regions are determined by finding locations where receiving strength is larger than $e^{-9}$ maximum strength. In other words, roughly 3 standard deviations away if Gaussian shape is assumed.
- fine mesh step size: By default, inside emission regions, step size in X is set to be $0.5\lambda_0$, with $\lambda_0$ the vacuum wave length. This can be tuned using `fine_coeff` argument. For example, `fine_coeff`=2 means steps size is half of the default.
- coarse mesh step size: Outside the emission regions, step size is set to be $5\lambda_0$. It is also changed with `fine_coeff` argument. So the ratio between fine and coarse step size is always 0.1 .

In [ ]:
ece.auto_adjust_mesh(fine_coeff=1)

In [ ]:
ece.X1D.shape

In [ ]:
plt.plot(ece.X1D)
plt.xlabel('array indices')
plt.ylabel('X(cm)')
plt.title('Auto mesh in X')

The total mesh points is less, but more focused in emission area. How about the performance?

In [ ]:
ece.diagnose(debug=True)

In [ ]:
ece.Te/keV

Now, the result is roughly 50 times more accurate with only 1.5 times mesh points! 

> It is recommended to always use `auto_adjust_mesh` before `diagnose`, or simply set `auto_patch=True` when calling `diagnose`.

### 1.3.3 Information about diagnosing locations

A useful information is where our signal comes from. `ECE2D` provides two ways to access this.
- view_point: gives the coordinates (Z, Y, X) of the highest receiving strength, which is presumabily the best estimation of the actually center of the emission signal.
- diag_x: gives a detailed information of locations in X where significant contributions of emission power are from. list of centers and *half widths* are returned.

In [ ]:
ece.view_point

In [ ]:
xs, dxs = ece.diag_x
print(xs, dxs)

As we can see, the center of the emission region may not be the same as the strongest emission location. This is due to the uneven shape of the spot. The *half width* is a very rough estimation of the $1/e$ width of the spot, but can give us a sense of the resolution in X.

## 1.4 Multi-channel ECE Imaging

**TO BE COMPLETED**

# 2. Benchmarks and Error Assessment

In this section, we'll be showing effects of higher harmonics, as well as the sensibility of the diagnostic accuracy on mesh resolution.

## 2.1 Higher harmonic effects

Higher harmonic may come in when temperature is high and relativistic downshift effect is significant. In our test case, 3rd harmonic resonance has a finite absorption layer at the outer edge of plasma.

In [ ]:
ece_3rd = rcp.ECE2D(plasma=p2d_uni, detector=detector, polarization='X', max_harmonic=3, max_power=3, 
                weakly_relativistic=True, isotropic=True)

In [ ]:
ece_3rd.set_coords([Z1D, Y1D, X1D])

In [ ]:
ece_3rd.diagnose(debug=True, auto_patch=True)

We can now look at the emission pattern in X-Y plane

In [ ]:
emission_spot_3rd = ece_3rd.integrand_list[0]

In [ ]:
plt.contour(ece_3rd.X2D, ece_3rd.Y2D, np.real(emission_spot_3rd[0,:,:]), 20)
#plt.scatter(ece_3rd.X2D, ece_3rd.Y2D, c=np.real(emission_spot_3rd[0,:,:]), s=5, linewidth=0)

It shows that main contribution is actually from the edge 3rd harmonic resonance layer. Locations can be verified with `view_point` and `diag_x` attributes.

In [ ]:
ece_3rd.view_point

In [ ]:
ece_3rd.diag_x

This will significantly affect final result if we have a non-uniform electron temperature.

In [ ]:
tp.set_parameter2D(Te_0=10*keV, Te_shape='Hmode', ne_shape='Hmode')
p2d_hmode = tp.create_profile2D()
p2d_hmode.setup_interps()

In [ ]:
ece3_hmode = rcp.ECE2D(plasma=p2d_hmode, detector=detector, polarization='X', max_harmonic=3, max_power=3, 
                weakly_relativistic=True, isotropic=True )

In [ ]:
ece3_hmode.set_coords([Z1D, Y1D, X1D])
ece3_hmode.diagnose(auto_patch=True)

In [ ]:
ece3_hmode.Te/keV

The measured Te is much smaller than the value at the aimed 2nd harmonic layer location. The actual view location is at the top of the pedestal(~ 249cm).

In [ ]:
print('view_point:{0}\ndiag_x:{1}'.format(ece3_hmode.view_point, ece3_hmode.diag_x))

The local temperatures at the view point and the center of the spot are:

In [ ]:
p2d_hmode.get_Te([[0,0],[ece3_hmode.view_point[2], ece3_hmode.diag_x[0][0]]])/keV

This confirms our measured temperature is from this region.

## 2.2 Error analysis

First, let's take a look at the error dependency on step sizes. We'll use the 2nd harmonic version, and uniform plasma in this section.

In [ ]:
base = 4
power_list = np.arange(5)*0.5
err_list = np.empty_like(power_list, dtype='float')
for i,p in enumerate(power_list):
    step_yz = int(2**(base+p))
    Y1D = np.linspace(-20, 20, step_yz+1)
    Z1D = np.linspace(-20, 20, step_yz+1)
    ece.auto_adjust_mesh(fine_coeff=2**p/4)
    ece.set_coords([Z1D, Y1D, ece.X1D])
    # each time set_coords is called, auto_coords_adjusted flag will be reset. 
    # Here we'll manually set it to save some time.
    ece._auto_coords_adjusted = True
    ece.diagnose()
    err_list[i] = 1 - ece.Te/Te0 

In [ ]:
err_list

In [ ]:
from scipy.stats import linregress
slope, intercept, r, p, stderr = linregress(power_list*np.log(2), np.log(np.abs(err_list)))
x = power_list*np.log(2)
y = np.exp(slope*x + intercept)

In [ ]:
slope

In [ ]:
plt.semilogy(x, np.abs(err_list))
plt.semilogy(x, y, 'r')
plt.text(x[2], 3e-3, 'slope : {0:.3}'.format(slope))
plt.xlabel('C+log(N)')
plt.ylabel('log(abserr)')

The error drops as square of step size, just as expected.

<!--bibtex

@book{svelto2010principles,
  title={Principles of Lasers},
  author={Svelto, O.},
  isbn={9781441913029},
  lccn={2009940423},
  url={https://books.google.com/books?id=ioywRI5W2HEC},
  year={2010},
  publisher={Springer US},
  pages={153--155}
}

@misc{wiki:gaussian_beam,
   author = "Wikipedia",
   title = "Gaussian beam --- Wikipedia{,} The Free Encyclopedia",
   year = "2016",
   url = "https://en.wikipedia.org/w/index.php?title=Gaussian_beam&oldid=703704493",
   note = "[Online; accessed 17-March-2016]"
 }

-->

# References

<a name="cite-svelto2010principles"/><sup>[^](#ref-1) </sup>Svelto, O.. 2010. _Principles of Lasers_. [URL](https://books.google.com/books?id=ioywRI5W2HEC)

